In [31]:
import os
import pandas as pd
import numpy as np
from pybaseball import team_game_logs, cache

In [48]:
TEAMS = [
    "ARI", "ATL", "BAL", "BOS", "CHC", "CHW", "CIN", "CLE", "COL", "DET",
    "MIA", "HOU", "KCR", "LAA", "LAD", "MIL", "MIN", "NYM", "NYY", "OAK",
    "PHI", "PIT", "SDP", "SFG", "SEA", "STL", "TBR", "TEX", "TOR", "WSN"
]

START = 2010 
END = 2022

In [47]:
cache.enable()
test = team_game_logs(2012, 'MIA')

c:\Users\albri\AppData\Local\Programs\Python\Python312\Lib\site-packages\pybaseball\team_game_logs.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_html(str(table))[0]
c:\Users\albri\AppData\Local\Programs\Python\Python312\Lib\site-packages\pybaseball\team_game_logs.py:36: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data = data.apply(pd.to_numeric, errors="ignore")


In [57]:
def collect_logs(start, end):
    df = pd.DataFrame()
    for year in range(start, end):
        for team in TEAMS:
            if team == 'MIA' and year < 2012:
                team = 'FLA'
            print(year, team)
            new_df = team_game_logs(year, team)
            new_df['team'] = team
            new_df['year'] = year
            if not df.empty:
                df = pd.concat([df, new_df])
            else:
                df = new_df
    return df
            
    

In [59]:
if os.path.exists("game_logs.csv"):
    games = pd.read_csv("game_logs.csv", index_col=0)
else:
    games = collect_logs(START, END)
    games.to_csv("game_logs.csv")

2010 ARI
2010 ATL
2010 BAL
2010 BOS
2010 CHC
2010 CHW
2010 CIN
2010 CLE
2010 COL
2010 DET
2010 FLA
2010 HOU
2010 KCR
2010 LAA
2010 LAD
2010 MIL
2010 MIN
2010 NYM
2010 NYY
2010 OAK
2010 PHI
2010 PIT
2010 SDP
2010 SFG
2010 SEA
2010 STL
2010 TBR
2010 TEX
2010 TOR
2010 WSN
2011 ARI
2011 ATL
2011 BAL
2011 BOS
2011 CHC
2011 CHW
2011 CIN
2011 CLE
2011 COL
2011 DET
2011 FLA
2011 HOU
2011 KCR
2011 LAA
2011 LAD
2011 MIL
2011 MIN
2011 NYM
2011 NYY
2011 OAK
2011 PHI
2011 PIT
2011 SDP
2011 SFG
2011 SEA
2011 STL
2011 TBR
2011 TEX
2011 TOR
2011 WSN
2012 ARI
2012 ATL
2012 BAL
2012 BOS
2012 CHC
2012 CHW
2012 CIN
2012 CLE
2012 COL
2012 DET
2012 MIA
2012 HOU
2012 KCR
2012 LAA
2012 LAD
2012 MIL
2012 MIN
2012 NYM
2012 NYY
2012 OAK
2012 PHI
2012 PIT
2012 SDP
2012 SFG
2012 SEA
2012 STL
2012 TBR
2012 TEX
2012 TOR
2012 WSN
2013 ARI
2013 ATL
2013 BAL
2013 BOS
2013 CHC
2013 CHW
2013 CIN
2013 CLE
2013 COL
2013 DET
2013 MIA
2013 HOU
2013 KCR
2013 LAA
2013 LAD
2013 MIL
2013 MIN
2013 NYM
2013 NYY
2013 OAK
2013 PHI
2

In [111]:
import datetime
from time import strptime

In [99]:
games.head(1)['Date']

0    Apr 5
Name: Date, dtype: object

In [124]:
def edit_date(game):
    og_date = game['Date'].strip().replace('susp', '')
    og_date = og_date.split(' ')

    first = 0
    if len(og_date) > 2 and og_date[2] == '(2)':
        first = 1


    game['Date'] = datetime.datetime(game['year'], strptime(og_date[0], '%b').tm_mon, int(og_date[1]), hour=first)
    
    return game

games = games.apply(edit_date, axis=1)

In [123]:
def game_results(game):
    result = game['Rslt'].split(',')
    game['WIN'] = True if result[0] == 'W' else False

    result = result[1].split('-')
    game['runs_for'] = int(result[0])
    game['runs_ag'] = int(result[1])
    
    
    return game

games = games.apply(game_results, axis=1)

In [140]:
def add_target(team):
    team['TARGET'] = team['WIN'].shift(-1)
    return team

games = games.groupby(['team', 'year'], group_keys=False).apply(add_target)

C:\Users\albri\AppData\Local\Temp\ipykernel_31872\4279141825.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  games = games.groupby(['team', 'year'], group_keys=False).apply(add_target)


In [126]:
game_copy = games.copy()

In [139]:
games = games.reset_index(drop=True)
games

,Game,Date,Home,Opp,Rslt,PA,AB,R,H,2B,...,OPS,LOB,NumPlayers,Thr,OppStart,team,year,WIN,runs_for,runs_ag
0,1,2010-04-05,True,SDP,"W,6-3",36,33,6,8,0,...,0.730,6,12,R,J.Garland(38),ARI,2010,True,6,3
1,2,2010-04-06,True,SDP,"L,3-6",39,34,3,7,1,...,0.635,9,17,R,C.Young(72),ARI,2010,False,3,6
2,3,2010-04-07,True,SDP,"W,5-3",32,31,5,9,1,...,0.757,3,16,R,K.Correia(48),ARI,2010,True,5,3
3,4,2010-04-09,True,PIT,"W,9-1",39,36,9,14,2,...,0.813,6,14,R,C.Morton(15),ARI,2010,True,9,1
4,5,2010-04-10,True,PIT,"L,3-6",34,29,3,5,2,...,0.776,4,14,L,Z.Duke(61),ARI,2010,False,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55243,158,2021-09-28,False,COL,"L,1-3",35,30,1,7,2,...,0.755,7,12,L,K.Freeland(57),WSN,2021,False,1,3
55244,159,2021-09-29,False,COL,"L,5-10",41,34,5,9,2,...,0.755,9,19,R,P.Lambert(20),WSN,2021,False,5,10
55245,160,2021-10-01,True,BOS,"L,2-4",41,35,2,9,2,...,0.755,12,15,L,E.Rodríguez(60),WSN,2021,False,2,4
55246,161,2021-10-02,True,BOS,"L,3-5",37,31,3,5,1,...,0.754,7,18,R,T.Houck(75),WSN,2021,False,3,5


In [142]:
games_copy2 = games.copy()
games = games_copy2

In [143]:
del games['R']
del games['RBI']
del games['NumPlayers']
del games['ROE']
del games['HBP']
del games['CS']
del games['IBB']
del games['BB']

In [145]:
del games['PA']

In [146]:
# Statistics to calculate running averages for after removing specified columns
stats = ['AB', 'H', '2B', '3B', 'HR', 'SO', 'SH', 'SF', 'GDP', 'SB']

# Function to calculate running averages
def running_avg(group):
    return group.expanding().mean()

for stat in stats:
    games[f'avg_{stat}'] = games.groupby(['team', 'year'], group_keys=False)[stat].apply(running_avg)


games


,Game,Date,Home,Opp,Rslt,AB,H,2B,3B,HR,...,avg_AB,avg_H,avg_2B,avg_3B,avg_HR,avg_SO,avg_SH,avg_SF,avg_GDP,avg_SB
0,1,2010-04-05,True,SDP,"W,6-3",33,8,0,0,2,...,33.000000,8.000000,0.000000,0.000000,2.000000,8.000000,0.000000,0.000000,1.000000,1.000000
1,2,2010-04-06,True,SDP,"L,3-6",34,7,1,0,0,...,33.500000,7.500000,0.500000,0.000000,1.000000,8.000000,0.000000,0.000000,1.000000,0.500000
2,3,2010-04-07,True,SDP,"W,5-3",31,9,1,0,4,...,32.666667,8.000000,0.666667,0.000000,2.000000,8.000000,0.000000,0.000000,1.000000,0.333333
3,4,2010-04-09,True,PIT,"W,9-1",36,14,2,0,1,...,33.500000,9.500000,1.000000,0.000000,1.750000,8.000000,0.000000,0.000000,1.250000,0.500000
4,5,2010-04-10,True,PIT,"L,3-6",29,5,2,0,1,...,32.600000,8.600000,1.200000,0.000000,1.600000,7.400000,0.000000,0.200000,1.400000,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55243,158,2021-09-28,False,COL,"L,1-3",30,7,2,0,0,...,33.240506,8.594937,1.658228,0.126582,1.132911,7.968354,0.240506,0.189873,0.981013,0.354430
55244,159,2021-09-29,False,COL,"L,5-10",34,9,2,0,0,...,33.245283,8.597484,1.660377,0.125786,1.125786,7.981132,0.238994,0.188679,0.987421,0.352201
55245,160,2021-10-01,True,BOS,"L,2-4",35,9,2,0,2,...,33.256250,8.600000,1.662500,0.125000,1.131250,7.987500,0.237500,0.187500,0.981250,0.350000
55246,161,2021-10-02,True,BOS,"L,3-5",31,5,1,0,1,...,33.242236,8.577640,1.658385,0.124224,1.130435,8.012422,0.236025,0.192547,0.975155,0.347826


In [151]:
df = games.merge(games, left_on=['Date', 'Opp'], right_on=['Date', 'team'], suffixes=('', '_opp'))
cols_drop = ['team_opp', 'year_opp', 'WIN_opp', 'runs_for_opp', 'runs_ag_opp', 'TARGET_opp', 'Game_opp', 'Home_opp', 'Opp_opp', 'Rslt_opp']

# Drop the specified columns
df = df.drop(columns=cols_drop)

In [152]:
df

,Game,Date,Home,Opp,Rslt,AB,H,2B,3B,HR,SO,SH,SF,GDP,SB,BA,OBP,SLG,OPS,LOB,Thr,OppStart,team,year,WIN,runs_for,runs_ag,TARGET,avg_AB,avg_H,avg_2B,avg_3B,avg_HR,avg_SO,avg_SH,avg_SF,avg_GDP,avg_SB,AB_opp,H_opp,2B_opp,3B_opp,HR_opp,SO_opp,SH_opp,SF_opp,GDP_opp,SB_opp,BA_opp,OBP_opp,SLG_opp,OPS_opp,LOB_opp,Thr_opp,OppStart_opp,avg_AB_opp,avg_H_opp,avg_2B_opp,avg_3B_opp,avg_HR_opp,avg_SO_opp,avg_SH_opp,avg_SF_opp,avg_GDP_opp,avg_SB_opp
0,1,2010-04-05,True,SDP,"W,6-3",33,8,0,0,2,8,0,0,1,1,0.242,0.306,0.424,0.730,6,R,J.Garland(38),ARI,2010,True,6,3,False,33.000000,8.000000,0.000000,0.000000,2.000000,8.000000,0.000000,0.000000,1.000000,1.000000,32,5,2,0,2,6,0,0,0,0,0.156,0.156,0.406,0.563,2,R,D.Haren(71),32.000000,5.000000,2.000000,0.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000
1,2,2010-04-06,True,SDP,"L,3-6",34,7,1,0,0,8,0,0,1,0,0.224,0.307,0.328,0.635,9,R,C.Young(72),ARI,2010,False,3,6,True,33.500000,7.500000,0.500000,0.000000,1.000000,8.000000,0.000000,0.000000,1.000000,0.500000,37,10,1,1,2,10,0,0,0,3,0.217,0.260,0.464,0.724,8,R,E.Jackson(49),34.500000,7.500000,1.500000,0.500000,2.000000,8.000000,0.000000,0.000000,0.000000,1.500000
2,3,2010-04-07,True,SDP,"W,5-3",31,9,1,0,4,8,0,0,1,0,0.245,0.308,0.449,0.757,3,R,K.Correia(48),ARI,2010,True,5,3,True,32.666667,8.000000,0.666667,0.000000,2.000000,8.000000,0.000000,0.000000,1.000000,0.333333,34,7,2,0,1,10,0,0,1,0,0.214,0.250,0.427,0.677,5,R,I.Kennedy(51),34.333333,7.333333,1.666667,0.333333,1.666667,8.666667,0.000000,0.000000,0.333333,1.000000
3,4,2010-04-09,True,PIT,"W,9-1",36,14,2,0,1,8,0,0,2,1,0.284,0.342,0.470,0.813,6,R,C.Morton(15),ARI,2010,True,9,1,False,33.500000,9.500000,1.000000,0.000000,1.750000,8.000000,0.000000,0.000000,1.250000,0.500000,34,8,1,0,0,9,0,0,0,1,0.255,0.352,0.394,0.746,7,R,R.López(60),34.250000,8.750000,1.750000,0.000000,1.000000,7.000000,0.500000,0.250000,1.000000,0.750000
4,5,2010-04-10,True,PIT,"L,3-6",29,5,2,0,1,5,0,1,2,0,0.264,0.328,0.448,0.776,4,L,Z.Duke(61),ARI,2010,False,3,6,True,32.600000,8.600000,1.200000,0.000000,1.600000,7.400000,0.000000,0.200000,1.400000,0.400000,36,9,1,0,2,10,1,0,0,3,0.254,0.340,0.405,0.745,6,R,D.Haren(41),34.600000,8.800000,1.600000,0.000000,1.200000,7.600000,0.600000,0.200000,0.800000,1.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55247,158,2021-09-28,False,COL,"L,1-3",30,7,2,0,0,4,0,0,4,0,0.259,0.337,0.418,0.755,7,L,K.Freeland(57),WSN,2021,False,1,3,False,33.240506,8.594937,1.658228,0.126582,1.132911,7.968354,0.240506,0.189873,0.981013,0.354430,31,9,4,1,1,13,1,0,1,0,0.248,0.315,0.413,0.728,9,L,P.Corbin(58),33.331210,8.254777,1.707006,0.210191,1.127389,8.426752,0.299363,0.222930,0.605096,0.464968
55248,159,2021-09-29,False,COL,"L,5-10",34,9,2,0,0,10,0,0,2,0,0.259,0.337,0.418,0.755,9,R,P.Lambert(20),WSN,2021,False,5,10,False,33.245283,8.597484,1.660377,0.125786,1.125786,7.981132,0.238994,0.188679,0.987421,0.352201,35,14,3,0,1,4,0,1,0,1,0.249,0.317,0.414,0.731,9,R,P.Espino(31),33.341772,8.291139,1.715190,0.208861,1.126582,8.398734,0.297468,0.227848,0.601266,0.468354
55249,160,2021-10-01,True,BOS,"L,2-4",35,9,2,0,2,9,0,0,0,0,0.259,0.337,0.418,0.755,12,L,E.Rodríguez(60),WSN,2021,False,2,4,False,33.256250,8.600000,1.662500,0.125000,1.131250,7.987500,0.237500,0.187500,0.981250,0.350000,33,7,0,0,2,4,0,0,1,0,0.261,0.328,0.448,0.776,9,L,J.Rogers(42),33.893750,8.837500,2.050000,0.137500,1.343750,8.493750,0.062500,0.262500,0.612500,0.250000
55250,161,2021-10-02,True,BOS,"L,3-5",31,5,1,0,1,12,0,1,0,0,0.258,0.337,0.417,0.754,7,R,T.Houck(75),WSN,2021,False,3,5,False,33.242236,8.577640,1.658385,0.124224,1.130435,8.012422,0.236025,0.192547,0.975155,0.347826,33,7,1,1,2,14,0,0,1,0,0.260,0.328,0.448,0.777,8,R,J.Gray(67),33.888199,8.826087,2.043478,0.142857,1.347826,8.527950,0.062112,0.260870,0.614907,0.248447
